In [ ]:
## Sensitivity Analysis for Phenolic-Coal Crosslinking Project
# The aim is to attempt using the four settings below for sensitivity analysis:
        # Factor prioritisation** - which parameters are most influential?
        # Factor fixing** - which parameters can we ignore?
        # Factor mapping - which inputs matter for just this space in the model output?
        # (Metamodelling) - which parameters can I use to model my model?

# Questions to answer, Can we determine;
        # 1. What is the sensitivity of the model to the input parameters?
        # 2. Which parameters are most influential?
        # 3. Which parameters can be ignored?
        # 4. Which parameters can be mapped to a reduced space?
        # 5. Which parameters can be used to model the model?

# The aim is to attempt these 2 approaches:
    # Local approaches
    # Global approaches